In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('../data/synthetic_clinical_logs.txt', sep='|', names=['Patient name', 'Temp (celcius)', 'BP', 'Notes'])

In [60]:
df = pd.DataFrame(df)
df.head(20)

,Patient name,Temp (celcius),BP,Notes
0,Patient: Erica Kennedy,Temp: 39.0°C,BP: 127/64,Notes: Mild cough
1,Patient: Kiara Fisher,Temp: --°C,BP: 127/62,Notes: headache and nausea
2,Patient: Douglas Benton,Temp: --°C,BP: 135/85,Notes: complained of fatigue
3,Patient: William Maldonado,Temp: --°C,BP: not taken,Notes: complained of fatigue
4,Patient: Nicole Velazquez,Temp: 36.8°C,BP: 142/71,Notes: chest pain
5,Patient: Tyler Hinton,Temp: 38.1°C,BP: 109/92,Notes:
6,Patient: Terri Cook,Temp: 39.2°C,BP: 144/74,Notes: no complaints
7,Patient: Jason Wise,Temp: --°C,BP: 126/71,"Notes: ""shortness of breath"""
8,Patient: Jorge Harmon,Temp: --°C,BP: not taken,Notes: chest pain
9,Patient: Nicholas Walker,Temp: 39.7°C,BP: 119/78,Notes:
